In [ ]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
batch_size = 4
num_workers = 2
epochs = 4
learning_rate = 0.001
betas = (0.9, 0.999)
PATH = '../data/saved_model/CNN_net.pth'

In [ ]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

trainset = torchvision.datasets.FashionMNIST(root = '../data', train=True,
                                       download=True, transform=transform)

trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                         shuffle=True, num_workers=num_workers)

testset = torchvision.datasets.FashionMNIST(root = '../data', train=False,
                                       download=True, transform=transform)

testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                        shuffle=False, num_workers=num_workers)

classes = ('T-shirt/top', 'Trouser', 'Pullover', 'Dress',
          'Coat', 'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot')

def imshow(img):
    img = img / 2 + 0.5                        # unnormalize
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show


In [4]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        
        self.features = nn.Sequential(
            # x : 28 * 28 * 1 (FashionMnist)
            # https://github.com/zalandoresearch/fashion-mnist
            nn.Conv2d(1, 6, 5),
            nn.ReLU(inplace=True),
            # x : 24 * 24 * 6
            nn.MaxPool2d(2, 2),
            # x : 14 * 14 * 6
            nn.Conv2d(6, 16, 5),
            nn.ReLU(inplace=True),
            # x : 10 * 10 * 16
            nn.MaxPool2d(2, 2)
        )
        self.classifier = nn.Sequential(
            nn.Dropout(),
            nn.Linear(4 * 4 * 16, 120),
            nn.Dropout(),
            nn.Linear(120, 84),
            nn.Dropout(),
            nn.Linear(84, 10)
        )
    def forward(self, x):
        x = self.features(x)
        # https://pytorch.org/docs/stable/torch.html#torch.flatten
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        
        return x


net = CNN()

if torch.cuda.device_count() > 1:
    print("Let's use", torch.cuda.device_count(), "GPUs!")
    net = nn.DataParallel(net)
net.to(device)

CNN(
  (features): Sequential(
    (0): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (classifier): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=256, out_features=120, bias=True)
    (2): Dropout(p=0.5, inplace=False)
    (3): Linear(in_features=120, out_features=84, bias=True)
    (4): Dropout(p=0.5, inplace=False)
    (5): Linear(in_features=84, out_features=10, bias=True)
  )
)

In [5]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=learning_rate, betas=betas)

In [6]:
for epoch in range(epochs):
    for i, (images, labels) in enumerate(trainloader, 0):
        images = images.to(device)
        labels = labels.to(device)
        
        out = net(images)
        loss = criterion(out, labels)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if i % 1000 == 990:
            print("[{epoch}, {index}] loss: {loss}".format(epoch = epoch + 1, index = i + 1, loss = loss.item()))
print("Finished Training")


[1, 991] loss: 0.862112283706665
[1, 1991] loss: 0.6580991744995117
[1, 2991] loss: 0.14126253128051758
[1, 3991] loss: 0.4368342161178589
[1, 4991] loss: 0.45767736434936523
[1, 5991] loss: 0.61182701587677
[1, 6991] loss: 2.1416380405426025
[1, 7991] loss: 0.7698377370834351
[1, 8991] loss: 1.09584641456604
[1, 9991] loss: 1.0421218872070312
[1, 10991] loss: 0.2796977758407593
[1, 11991] loss: 0.6221855878829956
[1, 12991] loss: 0.29405903816223145
[1, 13991] loss: 0.5076323747634888
[1, 14991] loss: 0.47638750076293945
[2, 991] loss: 0.08655273914337158
[2, 1991] loss: 1.1924101114273071
[2, 2991] loss: 0.09851384162902832
[2, 3991] loss: 1.717237949371338
[2, 4991] loss: 0.1193915605545044
[2, 5991] loss: 2.1439764499664307
[2, 6991] loss: 0.9709827899932861
[2, 7991] loss: 0.754624605178833
[2, 8991] loss: 0.3867439031600952
[2, 9991] loss: 0.4707169532775879
[2, 10991] loss: 0.5942692756652832
[2, 11991] loss: 1.26173734664917
[2, 12991] loss: 0.27968430519104004
[2, 13991] loss:

Test

In [8]:
net.eval()

correct = 0
total = 0

with torch.no_grad():
    for i, (images, labels) in enumerate(testloader, 0):
        images = images.to(device)
        labels = labels.to(device)
        
        out = net(images)
        
        _, predicted = torch.max(out.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the 10000 test images: {}'.format(
    100 * correct / total))

Accuracy of the network on the 10000 test images: 83.82


In [9]:
torch.save(net.state_dict(), PATH)